# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 12:00AM,238205,10,CLI00284.2,"CLINUVEL, Inc.",Released
1,May 27 2022 12:00AM,238205,10,CLI00285.2,"CLINUVEL, Inc.",Released
2,May 27 2022 12:00AM,238206,10,Enova-10036,Emerginnova,Released
3,May 27 2022 12:00AM,238206,10,Enova-10037,Emerginnova,Released
4,May 27 2022 12:00AM,238206,10,Eme-99963,Emerginnova,Released
5,May 27 2022 12:00AM,238206,10,Enova-10039,Emerginnova,Released
6,May 26 2022 3:29PM,238204,19,TR-RG-05262022,"GCH Granules USA, Inc.",Released
7,May 26 2022 3:25PM,238203,19,DEX0006876,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,May 26 2022 3:25PM,238203,19,DEX0006877,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,May 26 2022 3:18PM,238201,18,6508314,Innova Softgel LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,238201,Released,1
37,238203,Released,2
38,238204,Released,1
39,238205,Released,2
40,238206,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238201,NaN,NaN,1.0
238203,NaN,NaN,2.0
238204,NaN,NaN,1.0
238205,NaN,NaN,2.0
238206,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238121,0.0,1.0,0.0
238126,0.0,0.0,1.0
238129,0.0,0.0,1.0
238133,0.0,8.0,3.0
238139,8.0,13.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238121,0,1,0
238126,0,0,1
238129,0,0,1
238133,0,8,3
238139,8,13,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238121,0,1,0
1,238126,0,0,1
2,238129,0,0,1
3,238133,0,8,3
4,238139,8,13,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238121,,1,
1,238126,,,1
2,238129,,,1
3,238133,,8,3
4,238139,8,13,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc."
2,May 27 2022 12:00AM,238206,10,Emerginnova
6,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc."
7,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
9,May 26 2022 3:18PM,238201,18,Innova Softgel LLC
10,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC"
11,May 26 2022 2:54PM,238199,19,ACG North America LLC
12,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.
19,May 26 2022 2:26PM,238195,10,ISDIN Corporation
30,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc.",,,2
1,May 27 2022 12:00AM,238206,10,Emerginnova,,,4
2,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc.",,,1
3,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
4,May 26 2022 3:18PM,238201,18,Innova Softgel LLC,,,1
5,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",,,1
6,May 26 2022 2:54PM,238199,19,ACG North America LLC,,,1
7,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,,,7
8,May 26 2022 2:26PM,238195,10,ISDIN Corporation,,,11
9,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc.",2,,
1,May 27 2022 12:00AM,238206,10,Emerginnova,4,,
2,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc.",1,,
3,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
4,May 26 2022 3:18PM,238201,18,Innova Softgel LLC,1,,
5,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",1,,
6,May 26 2022 2:54PM,238199,19,ACG North America LLC,1,,
7,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,7,,
8,May 26 2022 2:26PM,238195,10,ISDIN Corporation,11,,
9,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc.",2,,
1,May 27 2022 12:00AM,238206,10,Emerginnova,4,,
2,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc.",1,,
3,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
4,May 26 2022 3:18PM,238201,18,Innova Softgel LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc.",2.0,NaN,NaN
1,May 27 2022 12:00AM,238206,10,Emerginnova,4.0,NaN,NaN
2,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN,NaN
4,May 26 2022 3:18PM,238201,18,Innova Softgel LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:00AM,238205,10,"CLINUVEL, Inc.",2.0,0.0,0.0
1,May 27 2022 12:00AM,238206,10,Emerginnova,4.0,0.0,0.0
2,May 26 2022 3:29PM,238204,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,May 26 2022 3:25PM,238203,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0,0.0
4,May 26 2022 3:18PM,238201,18,Innova Softgel LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2858213,60.0,5.0,0.0
15,714518,3.0,13.0,8.0
16,952688,4.0,0.0,0.0
18,714493,5.0,8.0,0.0
19,1190971,8.0,1.0,0.0
20,1190854,36.0,2.0,0.0
21,476284,1.0,1.0,0.0
22,238126,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2858213,60.0,5.0,0.0
1,15,714518,3.0,13.0,8.0
2,16,952688,4.0,0.0,0.0
3,18,714493,5.0,8.0,0.0
4,19,1190971,8.0,1.0,0.0
5,20,1190854,36.0,2.0,0.0
6,21,476284,1.0,1.0,0.0
7,22,238126,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,5.0,0.0
1,15,3.0,13.0,8.0
2,16,4.0,0.0,0.0
3,18,5.0,8.0,0.0
4,19,8.0,1.0,0.0
5,20,36.0,2.0,0.0
6,21,1.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,15,Released,3.0
2,16,Released,4.0
3,18,Released,5.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
Released,60.0,3.0,4.0,5.0,8.0,36.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
2,Released,60.0,3.0,4.0,5.0,8.0,36.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
2,Released,60.0,3.0,4.0,5.0,8.0,36.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()